# AutoGluon Ensemble - California Housing Dataset

이 노트북은 AutoGluon을 사용하여 GBM, LGBM, XGBoost, CatBoost, Random Forest, Voting, Stacking 앙상블 모델을 튜닝하고 테스트합니다.
참조: `Plus_7_ensemble.ipynb`

==이 노트북은 해당 프롬프트로 작성하였음==

@Plus_7_ensemble.ipynb 파일을 참조해서 autogluon을 사용해서 GBM, LGBM, XGBoost, CatBoost, Random Forest,Voting, Stacking ensemble모델을 튜닝하고 테스트하는 코드를 생성해줘. 파일명은 Plus_7_ensemble_gemini.ipynb파일로 생성해줘.

In [ ]:
# %pip install autogluon
# Use '%pip install' instead of '!pip install'

In [ ]:
# %pip install tqdm

In [ ]:
# from tqdm import tqdm
# import time

# # 0부터 99까지 반복하며 진행 바(progressive bar)를 표시합니다.
# for i in tqdm(range(100)):
#     time.sleep(0.05)  # 작업을 시뮬레이션하기 위한 짧은 대기 시간

In [ ]:
# tqdm을 포함하고있음 (install -q 옵션 : 로그없이 설치)
# %pip install ipywidgets

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

np.set_printoptions(precision=4)

c:\Users\user\miniconda3\envs\DS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. 데이터 로드 및 전처리

In [2]:
data = fetch_california_housing()
df = pd.DataFrame(data['data'], columns=data['feature_names'])
df['MedHouseVal'] = data['target']

print("데이터 크기:", df.shape)
df.head()

데이터 크기: (20640, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## 2. 데이터 분리 (Train/Test Split)

In [3]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

print("Train size:", train_data.shape)
print("Test size:", test_data.shape)

Train size: (16512, 9)
Test size: (4128, 9)


## 3. AutoGluon 모델 학습

요구사항:
- 모델: GBM, LGBM (AutoGluon의 GBM이 LightGBM 기반), XGBoost, CatBoost, Random Forest
- 앙상블: Voting (WeightedEnsemble), Stacking

이를 위해 `fit()` 함수에 다음 파라미터를 설정합니다:
- `hyperparameters`: {'GBM':{}, 'CAT':{}, 'XGB':{}, 'RF':{}}
- `num_bag_folds`: 5 (Stacking을 위해 Bagging 필요)
- `num_stack_levels`: 1 (Stacking 층 1개 추가)
- `fit_weighted_ensemble`: True (Voting/Weighted Ensemble)

In [4]:
label = 'MedHouseVal'
eval_metric = 'r2' # 회귀 문제 평가 지표 (R-squared)

predictor = TabularPredictor(label=label, eval_metric=eval_metric).fit(
    train_data,
    presets='best_quality',
    time_limit=300, # 학습 시간 제한 (초, 필요시 조정)
    hyperparameters={
        'GBM': {},   # LightGBM
        'CAT': {},   # CatBoost
        'XGB': {},   # XGBoost
        'RF': {},    # Random Forest
    },
    num_bag_folds=5,
    num_stack_levels=1,
    fit_weighted_ensemble=True
)

No path specified. Models will be saved in: "AutogluonModels\ag-20260128_074843"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          22
Pytorch Version:    2.9.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       13.24 GB / 31.74 GB (41.7%)
Disk Space Avail:   371.38 GB / 454.67 GB (81.7%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` 

(_ray_fit pid=36840) [1000]	valid_set's l2: 0.207468	valid_set's r2: 0.835802


(_dystack pid=14572) 	0.848	 = Validation score   (r2)
(_dystack pid=14572) 	15.6s	 = Training   runtime
(_dystack pid=14572) 	0.73s	 = Validation runtime
(_dystack pid=14572) Fitting model: RandomForest_BAG_L1 ... Training model for up to 4.26s of the 20.63s of remaining time.
(_dystack pid=14572) 	Fitting 1 model on all data (use_child_oof=True) | Fitting with cpus=22, gpus=0, mem=0.1/13.6 GB
(_dystack pid=14572) 	0.8087	 = Validation score   (r2)
(_dystack pid=14572) 	9.02s	 = Training   runtime
(_dystack pid=14572) 	1.88s	 = Validation runtime
(_dystack pid=14572) Fitting model: WeightedEnsemble_L2 ... Training model for up to 49.10s of the 8.81s of remaining time.
(_dystack pid=14572) 	Fitting 1 model on all data | Fitting with cpus=22, gpus=0, mem=0.0/13.3 GB
(_dystack pid=14572) 	Ensemble Weights: {'LightGBM_BAG_L1': 0.882, 'RandomForest_BAG_L1': 0.118}
(_dystack pid=14572) 	0.8487	 = Validation score   (r2)
(_dystack pid=14572) 	0.04s	 = Training   runtime
(_dystack pid=14572) 

## 4. 모델 성능 평가 (Leaderboard)

In [5]:
# 테스트 데이터로 성능 평가
leaderboard = predictor.leaderboard(test_data)
leaderboard

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.859710,0.849392,r2,1.342480,1.283113,36.501383,1.342480,1.283113,36.501383,1,True,1
1,WeightedEnsemble_L3,0.859059,0.855010,r2,3.546065,6.694606,110.904022,0.007015,0.001513,0.164706,3,True,8
2,WeightedEnsemble_L2,0.858084,0.854173,r2,1.535521,1.346352,64.224756,0.005642,0.002482,0.127362,2,True,4
3,LightGBM_BAG_L2,0.857942,0.851327,r2,2.691110,3.422693,99.644341,0.169209,0.109071,21.246703,2,True,5
4,CatBoost_BAG_L2,0.857841,0.853739,r2,2.565752,3.340354,85.654304,0.043851,0.026733,7.256666,2,True,7
5,RandomForest_BAG_L2,0.854320,0.845424,r2,3.495199,6.666360,103.482651,0.973299,3.352739,25.085013,2,True,6
6,CatBoost_BAG_L1,0.851933,0.850459,r2,0.187398,0.060757,27.596010,0.187398,0.060757,27.596010,1,True,3
7,RandomForest_BAG_L1,0.808232,0.812024,r2,0.992022,1.969751,14.300245,0.992022,1.969751,14.300245,1,True,2


## 5. 최종 평가 결과

In [6]:
performance = predictor.evaluate(test_data)
print("최종 평가 결과:", performance)

최종 평가 결과: {'r2': 0.8590589713968664, 'root_mean_squared_error': np.float64(-0.4297561974934785), 'mean_squared_error': -0.18469038928405368, 'mean_absolute_error': -0.27786929070012745, 'pearsonr': 0.9269482070583237, 'median_absolute_error': -0.1766859264373779}
